In [1]:
def generate_instance(num_rows_columns, I, J, seed=42):
    """
    Generate I demand points and J candidate facility sites from a grid of size num_rows_columns x num_rows_columns.
    Returns: (demand_points, candidate_sites)
    """
    if seed is not None:
        random.seed(seed)
    
    # すべての格子点を生成（1始まり）
    all_points = [(x, y) for x in range(1, num_rows_columns + 1)
                         for y in range(1, num_rows_columns + 1)]
    
    # ランダムにシャッフル
    random.shuffle(all_points)
    
    # 十分な点があるか確認
    assert I + J <= len(all_points), "Grid is too small for given I and J."
    
    demand_points = all_points[:I]
    candidate_sites = all_points[I:I+J]
    
    return demand_points, candidate_sites


In [2]:
# 修正後のユークリッド距離の計算関数
def compute_distances(demand_points, candidate_sites):
    D, J = len(demand_points), len(candidate_sites)  # ここで D, J を定義
    distances = np.zeros((D, J))
    for d in range(D):
        for j in range(J):
            distances[d, j] = np.sqrt(
                (demand_points[d][0] - candidate_sites[j][0]) ** 2
                + (demand_points[d][1] - candidate_sites[j][1]) ** 2
            )
    return distances


In [3]:
def compute_wij_matrix(distances, alpha=0, beta=0.1):
    wij_matrix = np.exp(alpha - beta * distances)
    return wij_matrix

In [4]:
def compute_Ui_L(wij_matrix, J_L):
    """
    リーダーの既存施設による U_i^L を計算する関数

    Parameters:
        wij_matrix (np.array): D × J の w_ij の重み行列
        J_L (set): リーダーが既に持っている施設のインデックス集合

    Returns:
        np.array: 各需要点 i に対する U_i^L のベクトル
    """
    D, _ = wij_matrix.shape  # D: 需要点の数, J: 候補施設の数

    # J_L が空なら影響はゼロ
    if not J_L:
        return np.zeros(D)

    # 各需要点 i に対して、リーダーの施設 j ∈ J_L からの重みを合計する
    utility_vector = np.zeros(D)
    for j in J_L:
        # 列 j は、施設 j が各需要点に与える重み
        utility_vector += wij_matrix[:, j]

    return utility_vector


def compute_Ui_F(wij_matrix, J_F):
    """
    リーダーの既存施設による U_i^L を計算する関数

    Parameters:
        wij_matrix (np.array): D × J の w_ij の重み行列
        J_L (set): リーダーが既に持っている施設のインデックス集合

    Returns:
        np.array: 各需要点 i に対する U_i^L のベクトル
    """
    D, _ = wij_matrix.shape  # D: 需要点の数, J: 候補施設の数

    # J_L が空なら影響はゼロ
    if not J_F:
        return np.zeros(D)

    # 各需要点 i に対して、リーダーの施設 j ∈ J_L からの重みを合計する
    utility_vector = np.zeros(D)
    for j in J_F:
        # 列 j は、施設 j が各需要点に与える重み
        utility_vector += wij_matrix[:, j]

    return utility_vector


## 以下アルゴリズム

### 分子分母の計算

In [ ]:
def compute_Ai(x, y, w_ij, Ui_L):
    return np.array([
        Ui_L[i] + sum(w_ij[i][j] * (-y[j] * x[j]**2 + (1 + y[j]) * x[j]) for j in range(len(x)))
        for i in range(len(Ui_L))
    ])

def compute_Bi(x, y, w_ij, Ui_L, Ui_F):
    return np.array([
        Ui_L[i] + Ui_F[i] + sum(w_ij[i][j] * ((1 - y[j]) * x[j] + y[j]) for j in range(len(x)))
        for i in range(len(Ui_L))
    ])

### 変数

In [ ]:
def update_lambda(Ai, Bi):
    return np.sqrt(Ai) / Bi

def update_mu_amgm(Ai, Bi):
    """μ_i = 1 / (2 A_i B_i)  (式 22)"""
    return 1.0 / (2.0 * Ai * Bi)

### Quadratic Transform

In [1]:
def objective_x_max(x, y, lambdas, w_ij, Ui_L, Ui_F, h_i):
    Ai = compute_Ai(x, y, w_ij, Ui_L)
    Bi = compute_Bi(x, y, w_ij, Ui_L, Ui_F)
    total = sum(h_i[i] * (2 * lambdas[i] * np.sqrt(Ai[i]) - lambdas[i]**2 * Bi[i]) for i in range(len(h_i)))
    return total

def objective_y_amgm(y, x_fixed, w_ij, Ui_L, Ui_F, h_i):
    """
    h(y) = Σ_i h_i [ y_i A_i^2 + 1/(4 y_i B_i^2) ]   ← (符号そのまま最小化)
    """
    Ai = compute_Ai(x_fixed, y, w_ij, Ui_L)
    Bi = compute_Bi(x_fixed, y, w_ij, Ui_L, Ui_F)

    # 防御的にゼロ割りを避ける
    eps  = 1e-12
    term = h_i * (y * Ai**2 + 1.0 / (4.0 * np.maximum(eps, y) * Bi**2))
    return term.sum()

In [2]:
def minmax_solver(x0, y0, w_ij, Ui_L, Ui_F, h_i,
                  p,          # ← x の合計上限
                  r,          # ← y の合計上限
                  max_iter=1000, tol=1e-5):

    x, y = np.copy(x0), np.copy(y0)
    hist_Lhat, hist_dx, hist_dy = [], [], []

    for k in range(max_iter):
        print(f"\n🌸🌸🌸  Iteration {k+1}  🌸🌸🌸")

        # ---------- 1. λ 更新 🐣 --------------------------------------------
        Ai = compute_Ai(x, y, w_ij, Ui_L)
        Bi = compute_Bi(x, y, w_ij, Ui_L, Ui_F)
        lambdas = update_lambda(Ai, Bi)
        print("🐣  λ updated!")

        # ---------- 2. x 最大化 💡 -----------------------------------------
        res_x = maximize(
            objective_x_max, x,
            args=(y, lambdas, w_ij, Ui_L, Ui_F, h_i),
            bounds=[(0, 1)] * len(x),
            constraints=[make_sum_constraint(p)],   # Σx ≤ p
            options={'ftol': 1e-12, 'maxiter': 300}
        )
        x_new = res_x.x if res_x.success else x
        print(f"💡  x updated: {np.round(x_new, 3)}")

        # ---------- 3. μ 更新 🐥 ------------------------------------------
        Ai = compute_Ai(x_new, y, w_ij, Ui_L)
        Bi = compute_Bi(x_new, y, w_ij, Ui_L, Ui_F)
        mus = update_mu_amgm(Ai, Bi)
        print("🐥  μ updated!")

        # ---------- 4. y 最小化 🌼 ----------------------------------------
        upper_bounds_y = [max(0.0, 1.0 - x_new[j]) for j in range(len(y))]
        bounds_y = [(0.0, ub) for ub in upper_bounds_y]

        # y 最小化を AM-GM 版 objective に差し替え
        res_y = minimize(
            objective_y_amgm, y,                
            args=(x_new, w_ij, Ui_L, Ui_F, h_i),
            method='SLSQP',
            bounds=bounds_y,
            constraints=[make_sum_constraint(r)],
            options={'ftol': 1e-12, 'maxiter': 300}
        )
        y_new = res_y.x if res_y.success else y
        print(f"🌼  y updated: {np.round(y_new, 3)}")

        # ---------- 5. 丸めと履歴保存 🔧 -----------------------------------
        x_bin = round_to_binary_best_k(x_new, k=p)  # ← x は p 個まで 1
        y_bin = round_to_binary_best_k(y_new, k=r)  # ← y は r 個まで 1
        Lval  = compute_L(h_i, Ui_L, Ui_F, w_ij, x_bin, y_bin)

        hist_Lhat.append(Lval)
        hist_dx.append(np.linalg.norm(x_new - x))
        hist_dy.append(np.linalg.norm(y_new - y))

        print("🔧  After rounding:")
        print(f"     ➤ x (rounded): {x_bin}")
        print(f"     ➤ y (rounded): {y_bin}")
        print(f"📈  Objective L̂(x, y) = {Lval:.6f}")
        print(f"🔍  dx = {hist_dx[-1]:.2e}, dy = {hist_dy[-1]:.2e}")

        # ---------- 6. 収束判定 🎉 ----------------------------------------
        if hist_dx[-1] < tol and hist_dy[-1] < tol:
            print("🎉✨ 収束しました！Great job! ✨🎉")
            break

        x, y = x_new, y_new

    return x, y, hist_Lhat, hist_dx, hist_dy

In [3]:
def compute_Lhat(x, y, w_ij, Ui_L, Ui_F, h_i):
    Ai = Ui_L + (w_ij * (-y * x**2 + (1 + y) * x)).sum(axis=1)
    Bi = Ui_L + Ui_F + (w_ij * ((1 - y) * x + y)).sum(axis=1)
    return np.sum(h_i * Ai / Bi)

In [4]:
def make_sum_constraint(L):
    return {'type': 'ineq', 'fun': (lambda v, limit=L: limit - np.sum(v))}

In [5]:
def maximize(fun, x0, args=(), bounds=None, constraints=(), **kwargs):
    """minimize を使って fun を最大化する簡単ラッパー"""
    res = minimize(lambda v, *a: -fun(v, *a), x0,
                   args=args, bounds=bounds, constraints=constraints, **kwargs)
    res.fun = -res.fun   # 戻り値を「最大値」に修正
    return res

In [6]:
def compute_L(h_i, Ui_L, Ui_F, wij, x, y):
    """
    関数 L(x, y) を計算する

    Parameters:
        h (np.array): 需要点ごとの人口密度ベクトル (D,)
        Ui_L (np.array): 各需要点におけるリーダーの影響度 (D,)
        Ui_F (np.array): 各需要点におけるフォロワーの影響度 (D,)
        wij (np.array): 需要点と施設候補の重み行列 (D, J)
        x (np.array): リーダーが選択した施設配置 (J,)
        y (np.array): フォロワーが選択した施設配置 (J,)

    Returns:
        float: L(x, y) の計算結果
    """
    numerator = Ui_L + (wij @ x)  # 分子: リーダーの影響度 + 選択した施設の影響
    denominator = Ui_L + Ui_F + (wij @ np.maximum(x, y))  # 分母: 総合影響度

    return np.sum(h_i * (numerator / denominator))

In [7]:
def round_to_binary_best_k(v, k=2):
    """
    v : 1-D ndarray, 連続値ベクトル (0-1 区間)
    k : 1 を立てる最大個数
    ---------------------------
    上位 k 個の成分を 1 にして残りを 0 に丸める。
    """
    bin_v = np.zeros_like(v)
    if k > 0:
        top_idx = np.argsort(-v)[:k]     # 大きい順に k 個取り出す
        bin_v[top_idx] = 1.0
    return bin_v